Exception Handling
===============

One of the nice things about languages like Python is that, when something goes wrong, it doesn't have to mean the end of your program (unlike in C where you can spend hours trying to diagnose a segfault [heisenbug](https://en.wikipedia.org/wiki/Heisenbug)).

As a starting example of handling exceptions, consider a scenario where the user types in a number and you print its reciprocal.

In [ ]:
def reciprocal(msg='Enter a number'):
    num = int(input(f'{msg}: '))
    print(1/num)

reciprocal()

Of course, things get ugly if you enter in 0 or something which isn't a number at all.

In [ ]:
reciprocal('Enter 0')

In [ ]:
reciprocal('Enter "foo"')

It would be nice to anticipate invalid input and not let it crash the program.

In [ ]:
def reciprocal2(msg='Enter a number'):
    try:
        num = int(input(f'{msg}: '))
        print(1/num)
    except:
        print('Invalid input!')

reciprocal2('Enter 0')

That's better than the nasty traceback we saw before.  However, there's a big problem with this way of handling exceptions.

In [ ]:
def bad_reciprocal(msg='Enter a number'):
    try:
        num = int(input(f'{msg}: '))
        print(1/nun)
    except:
        print('Invalid input!')

bad_reciprocal()

What the?  How was 5 an invalid input?  Look more closely at the `print` statement.  I "accidentally" typed `nun` instead of `num`.  Since the `nun` variable didn't exist, Python raised a `NameError`.  However, we didn't see that because of our catch-all `except` block.  This leads to our first lesson: If possible, only catch those exception types you're expecting.

In [ ]:
def less_bad_reciprocal(msg='Enter a number'):
    try:
        num = int(input(f'{msg}: '))
        print(1/nun)
    except ValueError:
        print('Invalid input!')
    except ZeroDivisionError:
        print('Cannot divide by 0!')

In [ ]:
less_bad_reciprocal('Enter "foo"')

In [ ]:
less_bad_reciprocal('Enter 0')

You see?  Since we were specific about which errors we were anticipating, we were alerted to a typo in our code.

Let's fix the typo.

In [ ]:
def good_reciprocal(msg='Enter a number'):
    try:
        num = int(input(f'{msg}: '))
    except ValueError:
        print('Invalid input!')
        return
        
    try:
        print(1/num)
    except ZeroDivisionError:
        print('Cannot divide by 0!')

I also split the code into separate `try` blocks.  The reason is we have two lines of code each of which could produce a separate error.  Put another way, we're not anticipating that the `print` statement will raise a `ValueError` nor are we anticipating that the `input` line will raise a `ZeroDivisionError`.

try with else
--------------

Let's consider another example.

In [ ]:
def enter_a_number():
    try:
        int(input('Enter a number: '))
    except ValueError:
        print('That\'s not a number!')
        return
    
    print('Yup!  That\'s a number all right.')

Pretty simple but that `return` statement looks ugly.  To see what I mean, consider this function:

```python
def even_or_odd(num):
    if num % 2 == 0:
        print('Even')
        return

    print('Odd')
```

Sure the function works but clearly it should have been written as

```python
def even_or_odd(num):
    if num % 2 == 0:
        print('Even')
    else:
        print('Odd')
```

The `if`/`else` logic exists for a reason.

The `else` keyword can also be used with a `try` block.  The code under `else` will be executed if no exceptions were raised by the `try` block.  You can think of it as, "If an exception is raised, do this.  Otherwise, do that."

In [ ]:
def better_enter_a_number():
    try:
        int(input('Enter a number: '))
    except ValueError:
        print('That\'s not a number!')
    else:
        print('Yup!  That\'s a number all right.')

In [ ]:
better_enter_a_number()

finally
-------

There's also the `finally` keyword.  Code in a `finally` block will **always** be executed whether an exception was raised or not.

In [ ]:
def end_with_a_bow():
    try:
        num = int(input('Enter a number: '))
        print(f'Your number is half of {num * 2}.')
    finally:
        print('That\'s all, folks!')

In [ ]:
end_with_a_bow()

What do you think will be printed by this next cell?

In [ ]:
def weird_function(num):
    try:
        print('Returning num * 2')
        return num * 2
    finally:
        num *= 10
        print(f'num is now {num}.')

print(weird_function(3))

Did you guess right?  What happened here?  When the `return` statement was executed, the value `num * 2`, or 6, was placed in a sort of "return slot".  After that, the `finally` block was run which changed the value of `num`.  However, the value 6 was still in the "return slot".  Contrast this behavior with

In [ ]:
def less_weird_function(num):
    try:
        num *= 2
        print(f'Returning {num}.')
        return num
    finally:
        print('On second thought, I want to return chicken.')
        return 'chicken'

print(less_weird_function(3))

Here, the `finally` block overwrote the value stored in the "return slot".  By the way, that's just a name I made up.  Don't go searching StackOverflow or the [Python docs](docs.python.org) for "return slot".

Chained exceptions
-----------------------

There are occasions when some code will raise an exception and, though you'd like to propagate the error to the user, you'd like to change what type of error the user receives.  This comes up a lot when you've imported someone else's code into your project and they've got their own custom exception classes which your user won't understand.

Consider this contrived example:

In [ ]:
class SuperConfusingError(Exception):
    pass

def imported_function(num):
    if num == 3:
        raise SuperConfusingError('According to the laws set down by the Padishah Emperor Shaddam IV, the number 3 is unacceptable as it attracts sand worms.')
    return num * (num+1) // 2

def user_function(msg='Please enter a number'):
    num = int(input(f'{msg}: '))
    try:
        return imported_function(num)
    except SuperConfusingError:
        raise ValueError('3 is no good.')

user_function('Enter 3')

Well, that was no good.  I mean, we did replace the `SuperConfusingError` with a `ValueError` but we probably scared the pants off our user with that traceback.  Look at that message in the middle:

```text
During handling of the above exception, another exception occurred:
```

That makes it seem like two errors occurred.  Which one should the user pay attention to?  We want the user to focus on the `ValueError` since that's all he should care about.  That is, he only needs to know that 3 is an unacceptable input.  There are two ways to rectify this.

In [ ]:
def less_confusing_user_function(msg='Please enter a number'):
    num = int(input(f'{msg}: '))
    try:
        return imported_function(num)
    except SuperConfusingError as e:
        raise ValueError('3 is no good.') from e

less_confusing_user_function('Enter 3')

That's a *little* better.  If the user inspects the traceback, he'll see that there was in fact only one error but it was converted into another error.  We still have the problem of the super verbose traceback.  Does the user really need to see the internal guts of `imported_function`?  Probably not.  Let's try that again.

In [ ]:
def clean_user_function(msg='Please enter a number'):
    num = int(input(f'{msg}: '))
    try:
        return imported_function(num)
    except SuperConfusingError:
        raise ValueError('3 is no good.') from None

clean_user_function('Enter 3')

Ah, perfect!  Now the user only sees what he needs to see.  The `from None` in the `raise` line erases the prior traceback information.

Bare exceptions
-------------------

I mentioned earlier that you should be as specific as possible when catching exceptions.  However, what if, once again, you've got a function you've imported from some third-party library and the library's documentation doesn't tell you what kind of exceptions the function can raise?  You'd still like to catch any potential exceptions and perhaps log them to the screen.  Can we do a bare `except` in this special case?  That is, is this okay?

```python
try:
    return imported_function()
except:
    print('imported_function failed.')
    return None
```

Nope!  That's still unacceptable.  To put it clearly, you should never, ever, ever, ever, ever, **ever** use a bare `except`.  What you should do instead is

```python
try:
    return imported_function()
except Exception:
    print('imported_function failed.')
    return None
```

What's the difference?  After all, a bare `except` catches all errors and `except Exception` catches all errors, right?

Actually, that's not correct.  While almost all exceptions you encounter will be instances of the `Exception` class, there is a more general category.

In [ ]:
print(issubclass(Exception, BaseException))
print(issubclass(SystemExit, Exception))
print(issubclass(SystemExit, BaseException))

`BaseException` is the true base class of all exceptions.  However, you only want to catch exceptions which fall under `Exception`.  A bare `except` will catch instances of `BaseException`.  Why is this a problem?  What if you wanted to terminate your program under certain conditions?  Calling `exit` actually raises a `SystemExit` exception which, as we just saw, falls under `BaseException` but not `Exception`.  We probably don't want to stop the program from exiting if that's what it wants to do.

Long story short: **NEVER** do a bare `except`.  If you have to, do `except Exception`.